<a href="https://colab.research.google.com/github/roysaurav/Gold-Price-Prediction/blob/main/GoldModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Setup





In [ ]:
!pip install plotly==4.8

     |████████████████████████████████| 11.5MB 318kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [ ]:
model_training_start_date = '2015-01-01' #@param {type:"date"}
# Change training end date for out of sample test
model_training_end_date = '2020-12-31' #@param {type:"date"}
model_testing_start_date = '2020-01-01' #@param {type:"date"}
model_testing_end_date = '2020-12-31' #@param {type:"date"}

In [ ]:
#!pip install statsmodels==0.12.2

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from patsy import dmatrices
#from statsmodels.regression.rolling import RollingOLS
import plotly.express as px
from sklearn.metrics import mean_squared_error, r2_score
#pd.options.plotting.backend = "plotly"


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:

'''

from google.colab import drive
drive.mount("/content/drive")
'''

'\n\nfrom google.colab import drive\ndrive.mount("/content/drive")\n'

In [ ]:
#@title Data Prep

path = "/content/drive/MyDrive/CI Global/Project.xlsx"
# https://drive.google.com/file/d/16hS_2LyULBmsjLd8cCLkRJ-2WJA1U082/view
path = "https://drive.google.com/uc?export=download&id=16hS_2LyULBmsjLd8cCLkRJ-2WJA1U082"
df = pd.read_excel(path, sheet_name="Prepared Gold model data").set_index("Date").sort_index()
df = df[["DXY (USD)", "US Real rate (10Y)", "Gold Spot price"]]
# Convert them to annualised returns
df[["DXY (USD)", "Gold Spot price"]] = df[["DXY (USD)", "Gold Spot price"]].pct_change().mul(400).dropna()
df.head()

,DXY (USD),US Real rate (10Y),Gold Spot price
Date,,,
1967-03-31,NaN,1.70,NaN
1967-06-30,-0.524264,2.44,0.00000
1967-09-29,0.367801,2.56,0.00000
1967-12-29,6.844830,2.66,0.00000
1968-03-29,0.696286,1.82,2.86123


In [ ]:
#df.plot(backend="plotly")

# Correlation

In [ ]:
#@title Cross Sectional Correlations

fig = px.imshow(df.corr(), x= df.columns, y=df.columns, color_continuous_scale="RdYlGn", width=400, height=400)
fig.show()

In [ ]:
#@title Gold Correlations

# Correlation Period doesn't matter over longer periods of time
rolling_corr_df = df.rolling(4*10).corr().dropna().reset_index()
rolling_corr_df = rolling_corr_df[rolling_corr_df["level_1"] == "Gold Spot price"]
rolling_corr_df = rolling_corr_df.drop(columns=["Gold Spot price", "level_1"]).set_index("Date")
fig = rolling_corr_df.loc["1975-01-30":].plot( backend="plotly" ,title="Gold Correlation")

fig.show()

In [ ]:
#@title Interest Rates and US Dollar Relationship
rolling_corr_df = df.rolling(4*5).corr().dropna().reset_index()
rolling_corr_df = rolling_corr_df[rolling_corr_df["level_1"] == "US Real rate (10Y)"]
rolling_corr_df = rolling_corr_df.drop(columns=["US Real rate (10Y)", "level_1"]).set_index("Date")
rolling_corr_df[["DXY (USD)"]].plot(backend="plotly", title="Interest Rates and US Dollar Relationship")

In [ ]:
rolling_corr_df = df.rolling(4*5).corr().dropna().reset_index()
rolling_corr_df = rolling_corr_df[rolling_corr_df["level_1"] == "Gold Spot price"]
rolling_corr_df = rolling_corr_df.drop(columns=["Gold Spot price", "level_1"]).set_index("Date")
rolling_corr_df.plot(backend="plotly", title="Gold Correlation")

# Gold Returns vs DXY (USD) Linear Model




In [ ]:
X = df[model_training_start_date:model_training_end_date][[ "DXY (USD)"]]
Y = df[model_training_start_date:model_training_end_date][["Gold Spot price"]]
X = sm.add_constant(X) # adding a constant

mod = sm.OLS(Y,X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:        Gold Spot price   R-squared:                       0.340
Model:                            OLS   Adj. R-squared:                  0.312
Method:                 Least Squares   F-statistic:                     11.87
Date:                Fri, 19 Mar 2021   Prob (F-statistic):            0.00221
Time:                        18:43:27   Log-Likelihood:                -113.02
No. Observations:                  25   AIC:                             230.0
Df Residuals:                      23   BIC:                             232.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.7786      4.637      1.893      0.0

In [ ]:
#@title Model - Mean Square Error 

pred_x = df.loc[model_testing_start_date:model_testing_end_date][["DXY (USD)"]]#.to_list()


pred_x = sm.add_constant(pred_x) # adding a constant

pred_y = res.predict(pred_x)
actutal_y = df.loc[model_testing_start_date:model_testing_end_date][["Gold Spot price"]]
print('Mean Squared Error: %.2f' % mean_squared_error(actutal_y, pred_y))

Mean Squared Error: 506.92


In [ ]:
fig = px.scatter(df[model_training_start_date:], x="DXY (USD)", y="Gold Spot price", trendline="ols")
fig.update_yaxes(title_text='Gold Returns Annualised')
fig.update_yaxes(ticksuffix="%")
fig.update_xaxes(title_text='DXY Pct Changes')
fig.update_xaxes(ticksuffix="%")

results = px.get_trendline_results(fig)

#print(results.iloc[0][0].summary())
fig.show()


# Gold Returns vs US Real Rate Linear Model

In [ ]:
X = df[model_training_start_date:][[ "US Real rate (10Y)"]]
Y = df[model_training_start_date:][["Gold Spot price"]]
X = sm.add_constant(X) # adding a constant

mod = sm.OLS(Y,X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:        Gold Spot price   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                 -0.008
Method:                 Least Squares   F-statistic:                    0.8153
Date:                Fri, 19 Mar 2021   Prob (F-statistic):              0.376
Time:                        18:43:27   Log-Likelihood:                -117.78
No. Observations:                  25   AIC:                             239.6
Df Residuals:                      23   BIC:                             242.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 11.2858      6

In [ ]:
fig = px.scatter(df[model_training_start_date:], x="US Real rate (10Y)", y="Gold Spot price", trendline="ols")
fig.update_yaxes(title_text='Gold Returns Annualised')
fig.update_yaxes(ticksuffix="%")
fig.update_xaxes(title_text='US Real Rate')
fig.update_xaxes(ticksuffix="%")
fig.show()

# Gold Returns vs  DXY (USD) and US Real Rate Linear Model

In [ ]:
#y, X = dmatrices('Gold Spot price ~ US Real rate (10Y)	DXY (USD)', data=df, return_type='dataframe')
X = df[model_training_start_date:][[ "US Real rate (10Y)", 	"DXY (USD)"]]
Y = df[model_training_start_date:][["Gold Spot price"]]
X = sm.add_constant(X) # adding a constant

mod = sm.OLS(Y,X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:        Gold Spot price   R-squared:                       0.348
Model:                            OLS   Adj. R-squared:                  0.289
Method:                 Least Squares   F-statistic:                     5.877
Date:                Fri, 19 Mar 2021   Prob (F-statistic):            0.00902
Time:                        18:43:27   Log-Likelihood:                -112.87
No. Observations:                  25   AIC:                             231.7
Df Residuals:                      22   BIC:                             235.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 10.1085      5